In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 24
compression = 0.5
dropout_rate = 0.2

In [0]:
#!pip install PyDrive
#import os
#from pydrive.drive import GoogleDrive
#from pydrive.auth import GoogleAuth
#gauth = GoogleAuth()
#drive = GoogleDrive(gauth)
#file_list = drive.ListFile(
#    {'q': "'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk' in parents"}).GetList()
#(x_train, y_train), (x_test, y_test) = drive.GetList()
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

#from google.colab import files
#uploaded = files.upload()

In [6]:
'''
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


with open(fname, 'r') as f:
  print(f.read())
  '''

'\n!pip install -U -q PyDrive\nimport os\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\n# 1. Authenticate and create the PyDrive client.\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n\n# choose a local (colab) directory to store the data.\nlocal_download_path = os.path.expanduser(\'~/data\')\ntry:\n  os.makedirs(local_download_path)\nexcept: pass\n\n# 2. Auto-iterate using the query syntax\n#    https://developers.google.com/drive/v2/web/search-parameters\nfile_list = drive.ListFile(\n    {\'q\': "\'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk\' in parents"}).GetList()\n\nfor f in file_list:\n  # 3. Create & download by id.\n  print(\'title: %s, id: %s\' % (f[\'title\'], f[\'id\']))\n  fname = os.path.join(local_download_path, f[\'title\'])\n  print(\'downloading to {}\'.format(fname))\n  f

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
'''

datagen = ImageDataGenerator(rotation_range=90)
datagen.fit(x_train)
'''
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [13]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])

In [15]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 236s 5ms/step - loss: 1.7471 - acc: 0.3644 - val_loss: 1.8868 - val_acc: 0.3807
Epoch 2/50
19200/50000 [==========>...................] - ETA: 2:08 - loss: 1.3223 - acc: 0.5160

50000/50000 [==============================] - 223s 4ms/step - loss: 1.2300 - acc: 0.5513 - val_loss: 1.2699 - val_acc: 0.5689
Epoch 3/50
45568/50000 [==========================>...] - ETA: 18s - loss: 1.0051 - acc: 0.6387

50000/50000 [==============================] - 221s 4ms/step - loss: 0.9992 - acc: 0.6415 - val_loss: 1.5535 - val_acc: 0.5413
Epoch 4/50
50000/50000 [==============================] - 221s 4ms/step - loss: 0.8714 - acc: 0.6882 - val_loss: 1.2904 - val_acc: 0.6068
Epoch 5/50
 3328/50000 [>.............................] - ETA: 3:12 - loss: 0.7941 - acc: 0.7218

50000/50000 [==============================] - 221s 4ms/step - loss: 0.7731 - acc: 0.7266 - val_loss: 1.0981 - val_acc: 0.6389
Epoch 6/50
39424/50000 [======================>.......] - ETA: 43s - loss: 0.7032 - acc: 0.7519

50000/50000 [==============================] - 221s 4ms/step - loss: 0.7032 - acc: 0.7517 - val_loss: 1.1724 - val_acc: 0.6277
Epoch 7/50
50000/50000 [==============================] - 222s 4ms/step - loss: 0.6547 - acc: 0.7703 - val_loss: 0.7096 - val_acc: 0.7565
Epoch 8/50
 1792/50000 [>.............................] - ETA: 3:20 - loss: 0.5984 - acc: 0.7868

...

50000/50000 [==============================] - 221s 4ms/step - loss: 0.2476 - acc: 0.9122 - val_loss: 0.4969 - val_acc: 0.8489
Epoch 45/50
38656/50000 [======================>.......] - ETA: 46s - loss: 0.2387 - acc: 0.9141

50000/50000 [==============================] - 220s 4ms/step - loss: 0.2381 - acc: 0.9144 - val_loss: 0.5612 - val_acc: 0.8386
Epoch 46/50
50000/50000 [==============================] - 221s 4ms/step - loss: 0.2421 - acc: 0.9143 - val_loss: 0.5234 - val_acc: 0.8402
Epoch 47/50
 1536/50000 [..............................] - ETA: 3:19 - loss: 0.2256 - acc: 0.9102

50000/50000 [==============================] - 221s 4ms/step - loss: 0.2361 - acc: 0.9155 - val_loss: 0.5700 - val_acc: 0.8463
Epoch 48/50
38656/50000 [======================>.......] - ETA: 46s - loss: 0.2317 - acc: 0.9172

50000/50000 [==============================] - 221s 4ms/step - loss: 0.2350 - acc: 0.9167 - val_loss: 0.6114 - val_acc: 0.8270
Epoch 49/50
50000/50000 [==============================] - 221s 4ms/step - loss: 0.2286 - acc: 0.9183 - val_loss: 0.5440 - val_acc: 0.8471
Epoch 50/50
 1536/50000 [..............................] - ETA: 3:19 - loss: 0.2152 - acc: 0.9232

50000/50000 [==============================] - 221s 4ms/step - loss: 0.2297 - acc: 0.9184 - val_loss: 0.5613 - val_acc: 0.8435


In [16]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 21s 2ms/step
Test loss: 0.5612559288024902
Test accuracy: 0.8435


In [17]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')

In [19]:
print(x_train.size)

153600000
